In [109]:
# DASHBOARD VỚI 13 CHART + FILTER THỨ TRONG TUẦN
import dash
from dash import html, dcc, Input, Output
import pandas as pd
import plotly.express as px

# Load data
df = pd.read_csv("df_final.csv", parse_dates=["saledate"])
df_unique = df.drop_duplicates(subset="orderid").reset_index(drop=True)

# Filters
districts = sorted(df_unique['Quận/Huyện'].dropna().unique())
time_options = df_unique['Time of day'].dropna().unique()
weekday_options = df_unique['Weekday'].dropna().unique()

# App
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("🎬 Dashboard Doanh Thu Rạp Phim", style={
        'textAlign': 'center',
        'color': '#003049',
        'marginBottom': '20px'
    }),

    html.Div([
        html.Div([
            html.Label("Chọn Quận/Huyện:"),
            dcc.Dropdown(
                options=[{'label': d, 'value': d} for d in districts],
                id='district-filter', value=None, placeholder="Tất cả"
            )
        ], style={'width': '30%', 'display': 'inline-block', 'paddingRight': '10px'}),

        html.Div([
            html.Label("Chọn Thời gian trong ngày:"),
            dcc.Dropdown(
                options=[{'label': t, 'value': t} for t in time_options],
                id='time-filter', value=None, placeholder="Tất cả"
            )
        ], style={'width': '30%', 'display': 'inline-block', 'paddingRight': '10px'}),

        html.Div([
            html.Label("Chọn Thứ trong tuần:"),
            dcc.Dropdown(
                options=[{'label': w, 'value': w} for w in weekday_options],
                id='weekday-filter', value=None, placeholder="Tất cả"
            )
        ], style={'width': '30%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'flexWrap': 'wrap', 'marginBottom': '30px'}),

    html.Div([
        html.Div([dcc.Graph(id='revenue-bar')], style={'width': '33%', 'display': 'inline-block'}),
        html.Div([
    dcc.Graph(
        id='chart2',
        figure=px.bar(
            df_unique, x='Time of day', y='total',
            title='Doanh thu theo thời điểm trong ngày',
            labels={'total': 'Tổng doanh thu', 'Time of day': 'Thời điểm'},
            color='Time of day'
        ),
        style={'outline': 'none'}  
    )
], style={'width': '33%', 'display': 'inline-block'}),

        
        html.Div([dcc.Graph(id='chart3', figure=px.line(
            df_unique[df_unique['saledate'].dt.month == 5]
            .groupby(df_unique['saledate'].dt.day)['total'].sum().reset_index(),
            x='saledate', y='total', markers=True,
            labels={'saledate': 'Ngày', 'total': 'Tổng doanh thu'},
            title='Tổng doanh thu theo ngày trong tháng 5'
        ))], style={'width': '33%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'flexWrap': 'wrap'}),

    html.Div([
        html.Div([dcc.Graph(id='chart6', figure=px.bar(
            df_unique.groupby('Weekday')['total'].sum().reset_index().sort_values(by='total', ascending=False),
            x='Weekday', y='total',
            title='Doanh Thu Theo Các Ngày Trong Tuần'
        ))], style={'width': '33%', 'display': 'inline-block'}),

        html.Div([dcc.Graph(id='chart5', figure=px.bar(
            df_unique.groupby('Quận/Huyện')['total'].sum().reset_index().sort_values(by='total', ascending=False),
            x='Quận/Huyện', y='total',
            title='Doanh Thu Theo Quận/Huyện'
        ))], style={'width': '33%', 'display': 'inline-block'}),

        html.Div([dcc.Graph(id='chart4', figure=px.pie(
            df_unique.groupby('Quận/Huyện')['total'].sum().reset_index(),
            names='Quận/Huyện', values='total',
            title='Tỷ Lệ Doanh Thu Theo Quận/Huyện (Gradient Màu)'
        ))], style={'width': '33%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'flexWrap': 'wrap'}),

    html.Div([
        html.Div([dcc.Graph(id='chart7', figure=px.bar(
            df_unique.groupby('industry')['total'].sum().reset_index(),
            x='industry', y='total',
            title='Doanh Thu Theo Nghề Nghiệp'
        ))], style={'width': '33%', 'display': 'inline-block'}),

        html.Div([dcc.Graph(id='chart8', figure=px.pie(
            df_unique.groupby('industry')['total'].sum().reset_index(),
            names='industry', values='total',
            title='Doanh Thu Theo Ngành'
        ))], style={'width': '33%', 'display': 'inline-block'}),

        html.Div([dcc.Graph(id='chart9', figure=px.bar(
            df_unique.groupby('Age Group')['total'].sum().reset_index(),
            x='total', y='Age Group', orientation='h',
            title='Doanh Thu Theo Nhóm Khách Hàng'
        ))], style={'width': '33%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'flexWrap': 'wrap'}),

    html.Div([
        html.Div([dcc.Graph(id='chart10', figure=px.bar(
            df_unique.groupby(['Age Group', 'Gender'])['total'].sum().reset_index(),
            x='Age Group', y='total', color='Gender', barmode='group',
            title='Doanh Thu Theo Nhóm Tuổi và Giới Tính'
        ))], style={'width': '33%', 'display': 'inline-block'}),

        html.Div([dcc.Graph(id='chart11', figure=px.pie(
            df.explode('country').groupby('country')['total'].sum().reset_index(),
            names='country', values='total',
            title='Doanh Thu Theo Quốc Gia Phim'
        ))], style={'width': '33%', 'display': 'inline-block'}),

        html.Div([dcc.Graph(id='chart12', figure=px.bar(
            df.groupby('listed_in')['total'].sum().reset_index().sort_values(by='total', ascending=False),
            y='listed_in', x='total', orientation='h',
            title='Doanh Thu Theo Thể Loại Phim'
        ))], style={'width': '33%', 'display': 'inline-block'}),

        
    ], style={'display': 'flex', 'flexWrap': 'wrap'})
])

@app.callback(
    Output('revenue-bar', 'figure'),
    Input('district-filter', 'value'),
    Input('time-filter', 'value'),
    Input('weekday-filter', 'value')
)
def update_graph(district, time, weekday):
    filtered_df = df_unique.copy()
    if district:
        filtered_df = filtered_df[filtered_df['Quận/Huyện'] == district]
    if time:
        filtered_df = filtered_df[filtered_df['Time of day'] == time]
    if weekday:
        filtered_df = filtered_df[filtered_df['Weekday'] == weekday]

    fig = px.bar(
        filtered_df.groupby('saledate')['total'].sum().reset_index(),
        x='saledate', y='total',
        title='Tổng doanh thu theo ngày'
    )
    return fig

if __name__ == '__main__':
    app.run(debug=True)